## Start

In [63]:
datadir = '../data';
wkdir = '../results';

mk_cd_dir(wkdir, false);
imatlab_export_fig('print-png')

% loocv inputs

prepare_loocv = true

demo_loocv = true

demo_loocv_number_or_list = 'number' % 'number' or 'list'

demo_loocv_number_cmpds = 2

demo_loocv_list_cmpds = {'BRD-K04804440','BRD-K01507359','BRD-K87202646','BRD-K59853741', 'BRD-K27302037'} % Ciprofloxacin, Rifampin, Isoniazid, Q203, Thioacetazone

results_subdir_prefix = 'loocv_pcls/leave_out_cmpd_'

loocv_save_out = false % save tabular file for each treatment's PCL similarity score


% general inputs

rr_path = fullfile(datadir, 'replicate_correlation_all.txt')

fgr_path = fullfile(datadir, 'fraction_gr.txt')

% previous Spectral Clustering inputs

thrsh_rank = 20 % threshold for average pairwise rank of correlation across KABX to connect treatments as mutual nearest-neighbors

dynamic_thrsh_per_moa = false % if true then threshold is round(log(size of MOA) * thrsh_rank), otherwise identical threshold for every MOA

k_type = 'k_med_gap_den' % eigengap heuristic to take for estimating number of K clusters: k_num_zero, k_num_zero_plus_one, k_med_gap_den, k_gap_den (see create_laplacian_matrix.m for additional information)

if dynamic_thrsh_per_moa
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%dxlogsize_%s', thrsh_rank, k_type) % log(MOA size), i.e. the number of treatments/dsCGI profiles in the MOA
else
    outdir_name = sprintf('pcls_spectral_clustering_thrsh_rank_le%d_%s', thrsh_rank, k_type)
end

outdir = fullfile(wkdir, outdir_name)

mk_cd_dir(outdir, false);

pcls_moa_path = fullfile(outdir, 'pcls.gmt')

col_meta_all_filename = 'col_meta.txt'

col_meta_path = fullfile(wkdir, col_meta_all_filename)

kabx_annot_path = fullfile(datadir, 'kabx_annotations.xlsx')

unknown_target_description_values = {'','NA','NAN','NaN','whole cell only','unknown'} % in case of different processing filetypes and NA values stored in import/export

make_fig = true

out_tbl_savename = 'by_pcl_similarity_to_confidence_score_thresholds_from_training_on_kabx.txt'

opt_tbl_savename = 'by_pcl_optimal_similarity_score_thresholds_from_training_on_kabx.txt'

out_tbl_test_cmpd_savename = 'by_pcl_similarity_to_confidence_score_test_cmpd_results.txt'

high_confidence_pcl_confidence_score_thres = 1; % defined as such for uniformity and simplicity across all PCLs

%col_meta_path = '/idi/cgtb/morzech/idmp/combine_screens_456/col_meta.txt'

%kabx_annot_path = '/idi/cgtb/abond/annotation/KABX_for_manuscript_20240413.xlsx'

%pcl_stats_path = '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/verify_new_pcls_thrsh_rank_le20_k_med_gap_den/figures_corr_summary.txt'


prepare_loocv =

  logical

   1


demo_loocv =

  logical

   1


demo_loocv_number_or_list =

    'number'


demo_loocv_number_cmpds =

     2


demo_loocv_list_cmpds =

  1x5 cell array

  Columns 1 through 3

    {'BRD-K04804440'}    {'BRD-K01507359'}    {'BRD-K87202646'}

  Columns 4 through 5

    {'BRD-K59853741'}    {'BRD-K27302037'}


results_subdir_prefix =

    'loocv_pcls/leave_out_cmpd_'


loocv_save_out =

  logical

   0


rr_path =

    '../data/replicate_correlation_all.txt'


fgr_path =

    '../data/fraction_gr.txt'


thrsh_rank =

    20


dynamic_thrsh_per_moa =

  logical

   0


k_type =

    'k_med_gap_den'


outdir_name =

    'pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'


outdir =

    '../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'


pcls_moa_path =

    '../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcls.gmt'


col_meta_all_filename =

    'col_meta.txt'


col_meta_path =

    '../results/col_meta.txt'


k

In [2]:
exist(col_meta_path) > 0
exist(kabx_annot_path) > 0

exist(pcls_moa_path) > 0

%exist(pcl_stats_path) > 0

assert(exist(col_meta_path) > 0)
assert(exist(kabx_annot_path) > 0)

assert(exist(pcls_moa_path) > 0)

%assert(exist(pcl_stats_path) > 0)


ans =

  logical

   1


ans =

  logical

   1


ans =

  logical

   1



In [3]:
ls(outdir)

cluster_median_corr_n10819x1947.gctx  pcl_similarity_score_n10819x1140.gctx
ds_corr_n7229x10819.gctx	      pcls.gmt
pcl_and_moa_agree_n10819x1140.gctx



## Load column metadata

## Load KABX annotation

In [4]:
kabx_annot_path
kabx_annot = xls2table(kabx_annot_path,1,true);


kabx_annot_path =

    '../data/kabx_annotations.xlsx'

1 pert_id
2 pert_id_pubchem_cid
3 pert_iname
4 pert_class
5 pert_reference
6 target_description
7 target_description_long
8 target_process
9 target_pathway
10 canonical_smiles


## Merge column metadata with KABX annotation

## Parse gctx with the PCL similarity score (median correlation of treatment to each PCL's treatments)

In [5]:
g = glob(fullfile(outdir,'pcl_similarity_score_n*.gctx'))
s = parse_gctx(g{1});


g =

  1x1 cell array

    {'../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcl_similarity_score_n10819x1140.gctx'}

Reading ../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcl_similarity_score_n10819x1140.gctx [1140x10819]
Done [2.03 s].


In [6]:
s.chd


ans =

  24x1 cell array

    {'broad_id'               }
    {'canonical_smiles'       }
    {'frac_gr_le0_30'         }
    {'pert_class'             }
    {'pert_dose'              }
    {'pert_id'                }
    {'pert_id_pubchem_cid'    }
    {'pert_idose'             }
    {'pert_idose_tickl'       }
    {'pert_iname'             }
    {'pert_reference'         }
    {'pert_type'              }
    {'proj_broad_id'          }
    {'project_id'             }
    {'rcorr'                  }
    {'rcorr_rank'             }
    {'target_description'     }
    {'target_description_long'}
    {'target_pathway'         }
    {'target_process'         }
    {'trt_id'                 }
    {'x_csv_broad_id'         }
    {'x_median_total_count'   }
    {'x_subproject_id'        }



## Parse gctx with binary labels if KABX treatment belongs to MOA of a PCL, i.e, pcl_and_moa_agree

In [7]:
g = glob(fullfile(outdir,'pcl_and_moa_agree_n*.gctx'))
l = parse_gctx(g{1});


g =

  1x1 cell array

    {'../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcl_and_moa_agree_n10819x1140.gctx'}

Reading ../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcl_and_moa_agree_n10819x1140.gctx [1140x10819]
Done [1.42 s].


In [8]:
l.chd


ans =

  24x1 cell array

    {'broad_id'               }
    {'canonical_smiles'       }
    {'frac_gr_le0_30'         }
    {'pert_class'             }
    {'pert_dose'              }
    {'pert_id'                }
    {'pert_id_pubchem_cid'    }
    {'pert_idose'             }
    {'pert_idose_tickl'       }
    {'pert_iname'             }
    {'pert_reference'         }
    {'pert_type'              }
    {'proj_broad_id'          }
    {'project_id'             }
    {'rcorr'                  }
    {'rcorr_rank'             }
    {'target_description'     }
    {'target_description_long'}
    {'target_pathway'         }
    {'target_process'         }
    {'trt_id'                 }
    {'x_csv_broad_id'         }
    {'x_median_total_count'   }
    {'x_subproject_id'        }



## Annotate gct object with column metadata

## Select all profiles regardless of rcorr

In [9]:
%cidx = cell2mat(s.cdesc(:,s.cdict('rcorr')))>=0.0;
%sum(cidx)
disp(sprintf('Total number of compounds (pert_ids): %d', numel(unique(s.cdesc(:,s.cdict('pert_id'))))))
disp(sprintf('Total number of broad_ids: %d', numel(unique(s.cdesc(:,s.cdict('broad_id'))))))
disp(sprintf('Total number of proj_broad_ids: %d', numel(unique(s.cdesc(:,s.cdict('proj_broad_id'))))))
disp(sprintf('Total number of dsCGI profiles / treatments (cids): %d', numel(unique(s.cid))))
%disp(sprintf('Total number of compounds with high rcorr: %d', numel(unique(s.cdesc(cidx,s.cdict('broad_id'))))))

Total number of compounds (pert_ids): 611
Total number of broad_ids: 679
Total number of proj_broad_ids: 1141
Total number of dsCGI profiles / treatments (cids): 10819


## Select profiles from KABX

In [10]:
unknown_target_description_values

cidx = ~cellfun(@(x) any(strcmp(x, unknown_target_description_values)), s.cdesc(:, s.cdict('target_description')));
disp('Number of KABX MOA-annotated dsCGI profiles')
sum(cidx)
disp('Number of unknown, test dsCGI profiles')
sum(~cidx)


unknown_target_description_values =

  1x6 cell array

  Columns 1 through 5

    {0x0 char}    {'NA'}    {'NAN'}    {'NaN'}    {'whole cell only'}

  Column 6

    {'unknown'}

Number of KABX MOA-annotated dsCGI profiles

ans =

        9427

Number of unknown, test dsCGI profiles

ans =

        1392



In [11]:
% dsCGI profiles from unannotated test compounds NA, N/A, NAN, Not Applicable
sortrows(cell2table(tabulate(s.cdesc(~cidx,s.cdict('target_description'))),...
    'VariableNames',{'target_description','count','percent'}),1)

% dsCGI profiles from MOA-annotated KABX compounds
sortrows(cell2table(tabulate(s.cdesc(cidx,s.cdict('target_description'))),...
    'VariableNames',{'target_description','count','percent'}),1)


ans =

  1x3 table

    target_description    count    percent
    __________________    _____    _______

          {'NA'}          1392       100  


ans =

  81x3 table

                          target_description                           count    percent 
    _______________________________________________________________    _____    ________

    {'30S ribosome'                                               }     417       4.4235
    {'30S ribosome|InfB'                                          }      40      0.42431
    {'30S ribosome|tRNA synthesis'                                }      20      0.21216
    {'50S ribosome'                                               }     740       7.8498
    {'50S ribosome|FusA1'                                         }      40      0.42431
    {'ATP synthase'                                               }      38       0.4031
    {'AccD6'                                                      }      60      0.63647
    {'Alr'              

## Construct table with PCL similarity score and binary labels of if dsCGI profile matches each PCL's annotated MOA for fitting PCL confidence score mapping and finding high-confidence similarity score thresholds

In [12]:
ss = s


ss = 

  struct with fields:

        mat: [1140x10819 single]
        rid: {1140x1 cell}
        rhd: {7x1 cell}
      rdesc: {1140x7 cell}
        cid: {10819x1 cell}
        chd: {24x1 cell}
      cdesc: {10819x24 cell}
      rdict: [7x1 containers.Map]
      cdict: [24x1 containers.Map]
        src: '../results/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den/pcl_similarity_score_n10819x1140.gctx'
    version: 'GCTX1.0'
     h5path: '/0/DATA/0/matrix'
     h5name: 'unnamed'



In [13]:
% Get column and row metadata fields for table
col_meta_ss = cell2table([ss.cid,ss.cdesc],'VariableNames',['cid';ss.chd]);
col_meta_ss.target_description = any2str(col_meta_ss.target_description);
disp('Head of col_meta_ss tbl:');
disp(headt(col_meta_ss));
disp('Unique KABX MOA annotations including multi-target:');
unique(col_meta_ss.target_description)

row_meta_ss = cell2table([ss.rid,ss.rdesc],'VariableNames',['rid';ss.rhd]);
row_meta_ss.pcl_desc = any2str(row_meta_ss.pcl_desc);
disp('Head of row_meta_ss tbl:');
disp(headt(row_meta_ss));
disp('Unique PCL MOA annotations:');
unique(row_meta_ss.pcl_desc)

Head of col_meta_ss tbl:
    idx               field                                                                        value                                                          
    ___    ___________________________    ________________________________________________________________________________________________________________________

     1     {'cid'                    }    {'kabx2:BRD-A05468928-003-01-4:0.097500uM'                                                                             }
     2     {'broad_id'               }    {'BRD-A05468928-003-01-4'                                                                                              }
     3     {'canonical_smiles'       }    {'CC1CN(CCN1)c1cc2n(cc(C(O)=O)c(=O)c2cc1F)-c1ccc(F)cc1F'                                                               }
     4     {'frac_gr_le0_30'         }    {[                                                                                                                   

In [14]:
[a,b] = ind2sub(size(ss.mat),1:numel(ss.mat));

In [15]:
ss_tbl = [col_meta_ss(b,:),row_meta_ss(a,:)];
disp('Size of ss_tbl with column and row metadata fields from similarity score gct');
disp(size(ss_tbl));
disp('Attaching PCL similarity score');
ss_tbl.pcl_similarity_score = ss.mat(:);
disp(size(ss_tbl));

Size of ss_tbl with column and row metadata fields from similarity score gct
    12333660          33

Attaching PCL similarity score
    12333660          34



### Add binary dsCGI profile-PCL MOA agreement labels from pcl_and_moa_agree gct

In [16]:
l = ds_slice(l, 'cid', ss.cid, 'rid', ss.rid);

In [17]:
assert(isequal(ss.rid,l.rid), 'ss and l rows not in matching order')
assert(isequal(ss.cid,l.cid), 'ss and l rows not in matching order')

disp('Attaching dsCGI profile-PCL MOA agreement binary labels (1 for in-MOA / 0 for out-of-MOA)');
ss_tbl.pcl_and_moa_agree = l.mat(:);
disp(size(ss_tbl));

Attaching dsCGI profile-PCL MOA agreement binary labels (1 for in-MOA / 0 for out-of-MOA)
    12333660          35



### Rank PCLs (highest to lowest PCL similarity score) for every dsCGI profile / treatment

In [18]:
ss_rank = ss;
ss_rank.mat = rankorder(ss.mat,'descend');

In [19]:
assert(isequal(ss.rid,ss_rank.rid), 'ss and ss_rank rows not in matching order')
assert(isequal(ss.cid,ss_rank.cid), 'ss and ss_rank rows not in matching order')

In [20]:
disp('Attaching rank of PCLs for each dsCGI profile from high-to-low PCL similarity score');
ss_tbl.rank_pcl = ss_rank.mat(:);
disp(size(ss_tbl));

Attaching rank of PCLs for each dsCGI profile from high-to-low PCL similarity score
    12333660          36



### Find targets that can and cannot be confirmed 

In [21]:
all_targets = unique(ss_tbl.target_description);
length(all_targets)


ans =

    82



In [22]:
targets_with_match = unique(ss_tbl.target_description(ss_tbl.pcl_and_moa_agree == 1));
length(targets_with_match)


ans =

    79



In [23]:
targets_without_match = all_targets(~ismember(all_targets, targets_with_match))
length(targets_without_match)


targets_without_match =

  3x1 cell array

    {'Fba'            }
    {'NA'             }
    {'lipid esterases'}


ans =

     3



## Split training and test set

In [24]:
ss_tbl_train_and_test = ss_tbl;

disp('Size of train and test set tbl:');
disp(size(ss_tbl_train_and_test));

Size of train and test set tbl:
    12333660          36



In [25]:
test_cmpd_idx = ~ismember(ss_tbl_train_and_test.pert_id, kabx_annot.pert_id);

disp(sprintf('Number of rows that are test (non-KABX) treatments: %d', sum(test_cmpd_idx)));

Number of rows that are test (non-KABX) treatments: 1586880


In [26]:
ss_tbl_test_cmpd = ss_tbl_train_and_test(test_cmpd_idx, :);

disp('Size and head of test set tbl:');
disp(size(ss_tbl_test_cmpd));
disp(headt(ss_tbl_test_cmpd));

Size and head of test set tbl:
     1586880          36

    idx                  field                                                                                                   value                                                                                  
    ___    _________________________________    ________________________________________________________________________________________________________________________________________________________________________

     1     {'cid'                          }    {'tbda1:TCMDC-142807:0.781250uM'                                                                                                                                       }
     2     {'broad_id'                     }    {'TCMDC-142807'                                                                                                                                                        }
     3     {'canonical_smiles'             }    {'NA'                     

In [27]:
disp(sprintf('Number of rows that are training (KABX) treatments: %d', sum(~test_cmpd_idx)));

ss_tbl = ss_tbl_train_and_test(~test_cmpd_idx, :);

disp('Size and head of training (KABX) set tbl:');
disp(size(ss_tbl));
disp(headt(ss_tbl));

Number of rows that are training (KABX) treatments: 10746780
Size and head of training (KABX) set tbl:
    10746780          36

    idx                  field                                                                                                   value                                                                                  
    ___    _________________________________    ________________________________________________________________________________________________________________________________________________________________________

     1     {'cid'                          }    {'kabx2:BRD-A05468928-003-01-4:0.097500uM'                                                                                                                             }
     2     {'broad_id'                     }    {'BRD-A05468928-003-01-4'                                                                                                                                              }
  

In [28]:
assert(all(ismember(ss_tbl.pert_id, kabx_annot.pert_id)))

assert(size(ss_tbl, 1) + size(ss_tbl_test_cmpd, 1) == size(ss_tbl_train_and_test, 1))

## Run ROC analysis of PCL similarity score to MOA agreement over all dsCGI profiles-PCL pairs

## Check for any missing PCL similarity scores in the training KABX set

In [29]:
idx = ~isnan(ss_tbl.pcl_similarity_score);
disp(sprintf('Total number of KABX dsCGI profile-PCL pairs with PCL similarity score to be used in training: %d', sum(idx)));
ss_tbl_sele = ss_tbl(idx,:);

Total number of KABX dsCGI profile-PCL pairs with PCL similarity score to be used in training: 10746780


In [30]:
assert(sum(~idx) == 0, sprintf('There are KABX dsCGI profiles with missing PCL similarity scores: %d rows', sum(~idx)))
assert(sum(idx) == height(ss_tbl), sprintf('There are KABX dsCGI profiles with missing PCL similarity scores: %d rows', height(ss_tbl) - sum(idx)))

## Verify number of training (KABX) set compounds and treatments

In [31]:
disp(sprintf('Number of pert_ids in training (KABX) set ss_tbl: %d', numel(unique(ss_tbl_sele.pert_id))));
disp(sprintf('Number of broad_ids in training (KABX) set ss_tbl: %d', numel(unique(ss_tbl_sele.broad_id))));
disp(sprintf('Number of proj_broad_ids in training (KABX) set ss_tbl: %d', numel(unique(ss_tbl_sele.proj_broad_id))));
disp(sprintf('Number of treatments (cids) in training (KABX) set ss_tbl: %d', numel(unique(ss_tbl_sele.cid))));

Number of pert_ids in training (KABX) set ss_tbl: 437
Number of broad_ids in training (KABX) set ss_tbl: 505
Number of proj_broad_ids in training (KABX) set ss_tbl: 967
Number of treatments (cids) in training (KABX) set ss_tbl: 9427


In [32]:
ss_tbl_reset = ss_tbl_sele;

## Verify number of test (unknown compound) set compounds and treatments

In [33]:
disp(sprintf('Number of pert_ids in test set ss_tbl_test_cmpd: %d', numel(unique(ss_tbl_test_cmpd.pert_id))));
disp(sprintf('Number of broad_ids in test set ss_tbl_test_cmpd: %d', numel(unique(ss_tbl_test_cmpd.broad_id))));
disp(sprintf('Number of proj_broad_ids in test set ss_tbl_test_cmpd: %d', numel(unique(ss_tbl_test_cmpd.proj_broad_id))));
disp(sprintf('Number of treatments (cids) in test set ss_tbl_test_cmpd: %d', numel(unique(ss_tbl_test_cmpd.cid))));

Number of pert_ids in test set ss_tbl_test_cmpd: 174
Number of broad_ids in test set ss_tbl_test_cmpd: 174
Number of proj_broad_ids in test set ss_tbl_test_cmpd: 174
Number of treatments (cids) in test set ss_tbl_test_cmpd: 1392


## Verify number of PCL clusters in KABX set and test set tables matches gmt file from previous steps

In [34]:
pcl_list = unique(ss_tbl_sele.rid);
disp(sprintf('Number of PCLs in KABX set table ss_tbl_sele: %d', length(pcl_list)));

Number of PCLs in KABX set table ss_tbl_sele: 1140


In [35]:
moa_list = unique(ss_tbl_sele.pcl_desc);
disp(sprintf('Number of MOAs represented by PCLs in KABX set table ss_tbl_sele: %d', length(moa_list)));

Number of MOAs represented by PCLs in KABX set table ss_tbl_sele: 68


In [36]:
pcls = parse_gmt(pcls_moa_path);
pcls_tbl = struct2table(pcls);
disp('Head and size of pcls_tbl read in from PCL gmt file:');
headt(pcls_tbl)
size(pcls_tbl)

assert(height(pcls_tbl) == length(pcl_list), 'Number of PCLs in gmt file and KABX set table do not match')

Head and size of pcls_tbl read in from PCL gmt file:

ans =

  4x3 table

    idx      field               value         
    ___    _________    _______________________

     1     {'head' }    {'30S ribosome:group1'}
     2     {'desc' }    {'30S ribosome'       }
     3     {'len'  }    {[                  4]}
     4     {'entry'}    {4x1 cell             }


ans =

        1140           4



In [37]:
disp(sprintf('Number of PCLs in test set table ss_tbl_test_cmpd: %d', length(unique(ss_tbl_test_cmpd.rid))));

disp(sprintf('Number of MOAs represented by PCLs in test set table ss_tbl_test_cmpd: %d', length(unique(ss_tbl_test_cmpd.pcl_desc))));

assert(all(ismember(ss_tbl_test_cmpd.rid, pcl_list)), 'Number of PCLs in gmt file and test set table do not match')

Number of PCLs in test set table ss_tbl_test_cmpd: 1140
Number of MOAs represented by PCLs in test set table ss_tbl_test_cmpd: 68


## Iterate for each PCL

In [38]:
% save ss_tbl_reset as a copy of ss_tbl_sele which remains unchanged
% it will be used to subset and set ss_tbl_sele to the dsCGI profile similarity scores and MOA agreement labels for each specific PCL iteratively 
ss_tbl_reset = ss_tbl_sele;

In [81]:
tic

out_tbl = {};
opt_tbl = {};
out_tbl_test_cmpd = {};

high_confidence_ppv_thres = high_confidence_pcl_confidence_score_thres; % defined as such for uniformity and simplicity across all PCLs

num_pcls = numel(pcl_list)

%for i = 1:num_pcls
for i = 1:3

    loop_progress(i,num_pcls,100)

    choose_pcl = pcl_list(i)
    
    jj = find(ismember(pcls_tbl.head, choose_pcl))

    pcls(jj).entry
    
    this_pcl_size = pcls(jj).len

    % Subset to KABX set rows for given PCL %
    idx = strcmp(ss_tbl_reset.rid, choose_pcl);
    sum(idx)

    ss_tbl_sele = ss_tbl_reset(idx,:);
    size(ss_tbl_sele)

    choose_pcl_desc = ss_tbl_sele.pcl_desc(1);

    ss_tbl_sele = sortrows(ss_tbl_sele,{'pcl_similarity_score'},{'descend'});
    [unique_brds, idx] = unique(ss_tbl_sele.pert_id, 'stable'); % can be changed to broad_id, proj_broad_id, cid, etc. to consider top similarity scores at every compound detail level
    numel(idx)

    %min_rcorr = min(ss_tbl_sele.rcorr);

    % Commented out line would set highest_cmpd_min_pcl_similarity_score to similarity score of the KABX compound/treatment with the lowest similarity to the PCL to save compute time and storage
    % instead use -Inf to have confidence scores for similarity scores from all dsCGI profiles in the results file
    %highest_cmpd_min_pcl_similarity_score = min(ss_tbl_sele.pcl_similarity_score(idx))
    highest_cmpd_min_pcl_similarity_score = -Inf;

    %%% Take similarity scores to this PCL for all KABX dsCGI profiles %%%
    
    all_trt_score_tbl = ss_tbl_sele(:, {'cid','pert_id','broad_id','proj_broad_id','target_description','pcl_similarity_score','rcorr','pcl_and_moa_agree'});
    
    %all_trt_score_tbl.cmpd_index = repmat(nan,[size(all_trt_score_tbl, 1) 1]);

    %all_trt_scores = unique(table2array(all_trt_score_tbl(all_trt_score_tbl.pcl_similarity_score >= highest_cmpd_min_pcl_similarity_score, 'pcl_similarity_score')));
    all_trt_scores = all_trt_score_tbl.pcl_similarity_score(all_trt_score_tbl.pcl_similarity_score >= highest_cmpd_min_pcl_similarity_score);
    all_trt_labels = all_trt_score_tbl.pcl_and_moa_agree(all_trt_score_tbl.pcl_similarity_score >= highest_cmpd_min_pcl_similarity_score);

    % Take the highest similarity scoring treatments/dsCGI profiles for each KABX compound for initial binary classifier training to only count
    % compounds and not treatments as independent successes/true positives
    ss_tbl_sele = ss_tbl_sele(idx,:);
    
    %%% Section below uncommented would consider only highest the similarity scoring treatments for each KABX compound in the model fit%%% 
    %all_trt_scores = ss_tbl_sele.pcl_similarity_score;
    %all_trt_labels = ss_tbl_sele.pcl_and_moa_agree;
    
    %all_trt_score_tbl = ss_tbl_sele(:, {'cid','broad_id','target_description','pcl_similarity_score','rcorr','pcl_and_moa_agree'});
    
    %all_trt_score_tbl.cmpd_index = repmat(nan,[size(all_trt_score_tbl, 1) 1]);
    
    %%%

    num_correct = sum(ss_tbl_sele.pcl_and_moa_agree == 1)

    pcl_n = size(ss_tbl_sele, 1);

    tmp_opt_tbl = {};

    tmp_opt_tbl.pcl_rid = choose_pcl;
    tmp_opt_tbl.pcl_desc = choose_pcl_desc;
    tmp_opt_tbl.pcl_n = pcl_n;
    tmp_opt_tbl.pcl_n_correct = num_correct;
    tmp_opt_tbl.pcl_n_trt = length(all_trt_scores);
    tmp_opt_tbl.pcl_n_correct_trt = sum(all_trt_labels);

    if num_correct > 0

        train_cmpds = ss_tbl_sele.pert_id;
        
        train_pert_ids = ss_tbl_sele.pert_id;

        full_scores = ss_tbl_sele.pcl_similarity_score;

        assert(issorted(all_trt_scores, 'descend')) % high to low PCL similarity score 

        assert(issorted(full_scores, 'descend')) % high to low PCL similarity score

        %full_train_scores = max(full_scores)-full_scores;
        full_train_scores = full_scores;

        full_labels = ss_tbl_sele.pcl_and_moa_agree; % binary labels for if KABX compound has shared MOA with PCL (1 for in-MOA / 0 for out-of-MOA)

        % ROC analysis / fitting of binary classifier for current PCL to threshold similarity scores to it (based on KABX) and assign mechanism 
        % if an unknown compound's similarity score exceeds the threshold
        % binarized class labels (in-MOA vs out-of-MOA), PCL similarity scores, Positive class is 1 (in-MOA), Negative class is 0 (out-of-MOA)
        % prior probabilities are empirical (default) based on in-MOA/out-of-MOA frequencies of PCL's MOA in KABX
        % default [0 1;1 0] (since MATLAB R2022a) misclassification costs specified
        [fpr,tpr,t,auc,optrocpt] = perfcurve(full_labels,full_train_scores,1,'NegClass',0,'UseNearest','off','XVals','all','Cost',[0 1; 1 0]);
        auc
        optrocpt
        % PPV (Positive Predictive Value) taken as empirical PCL confidence scores at each observed PCL similarity score from KABX
        % full_train_scores contains the highest similarity scoring dsCGI profile each KABX compound (pert_id)
        [ppv,tpr,t] = perfcurve(full_labels,full_train_scores,1,'XCrit','ppv','NegClass',0,'UseNearest','off','XVals','all','Cost',[0 1; 1 0]);

        % Extend PCL similarity score to confidence score mapping to all KABX dsCGI profiles contained in all_trt_scores
        % through next-neighbor interpolation which assigns the PCL confidence score of an unobserved (not maximal) dsCGI profile for a KABX compound
        % to the PCL confidence score of the next most similar observed (maximal for the same or different KABX compound) dsCGI profile
        ppv_full = interp1(unique(full_scores, 'stable'), ppv(2:end), all_trt_scores,'next');
        
        % Ensure no missing values
        ppv_full = fillmissing(ppv_full, 'nearest');

        % Optimal threshold based on ROC analysis and default misclassification cost
        isOptimalPoint = ismember([fpr, tpr], optrocpt, 'rows');

        % Use find to return the indices of the rows that match the 1-by-2 array
        optimalPoint = find(isOptimalPoint(2:end)) + 1;

        tmp_opt_tbl.pcl_auc = auc;
        
        % high_confidence_ppv_thres set to 1; defined as such for uniformity and simplicity across all PCLs
            
        full_model_high_confidence_score_thres = all_trt_scores(max(find(single(ppv_full) >= single(high_confidence_ppv_thres))))
        
        if isempty(full_model_high_confidence_score_thres) % should never be the case given definition of PCL clusters as having some region with PCL confidence score = 1 but for completeness and error-checking
            full_model_high_confidence_score_thres = nan;
            
            full_model_next_score_thres = all_trt_score_tbl.pcl_similarity_score(1);
            full_model_next_score_ppv = 0;
            
            full_model_next_score_idx = 1;  
        else
            full_model_next_score_thres = all_trt_scores(max(find(single(ppv_full) >= single(high_confidence_ppv_thres)))+1)
            full_model_next_score_ppv = ppv_full(max(find(single(ppv_full) >= single(high_confidence_ppv_thres)))+1)
            
            full_model_next_score_idx = max(find(single(ppv_full) >= single(high_confidence_ppv_thres)))+1;
        end
        
        tmp_opt_tbl.pcl_high_confidence_similarity_score_threshold = full_model_high_confidence_score_thres;
        tmp_opt_tbl.pcl_high_confidence_ppv_thres = high_confidence_ppv_thres;

        if length(optimalPoint)>0

            optimalPPV = ppv(optimalPoint);

            full_model_opt_score_thres = all_trt_scores(max(find(single(ppv_full) >= single(optimalPPV))))
            
            high_confidence_ppv_thres
            optimalPPV

            %full_model_next_score_thres = all_trt_scores(max(find(single(ppv_full) >= single(optimalPPV)))+1)
            %full_model_next_score_ppv = ppv_full(max(find(single(ppv_full) >= single(optimalPPV)))+1)
            
            %full_model_next_score_idx = max(find(single(ppv_full) >= single(optimalPPV)))+1;

            tmp_opt_tbl.pcl_opt_similarity_score_threshold = full_model_opt_score_thres;
            tmp_opt_tbl.pcl_opt_ppv = ppv(optimalPoint);
            tmp_opt_tbl.pcl_opt_tpr = tpr(optimalPoint);
            tmp_opt_tbl.pcl_opt_fpr = fpr(optimalPoint);
        else

            optimalPPV = nan;

            full_model_opt_score_thres = nan
            
            high_confidence_ppv_thres
            optimalPPV

            %full_model_next_score_thres = nan
            %full_model_next_score_ppv = nan
            
            %full_model_next_score_idx = 1;
            
            tmp_opt_tbl.pcl_opt_similarity_score_threshold = nan;
            tmp_opt_tbl.pcl_opt_ppv = nan;
            tmp_opt_tbl.pcl_opt_tpr = nan;
            tmp_opt_tbl.pcl_opt_fpr = nan;
        end

    else % should never be the case that there no positive samples (KABX dsCGI profiles with matching MOA to the PCL) for any PCL but for completeness and error-checking
        ppv_full = repmat(nan,[size(all_trt_score_tbl, 1) 1]);
        
        full_model_high_confidence_score_thres = nan;
            
        full_model_next_score_thres = all_trt_score_tbl.pcl_similarity_score(1);
        full_model_next_score_ppv = 0;

        full_model_next_score_idx = 1;
        
        tmp_opt_tbl.pcl_high_confidence_similarity_score_threshold = nan;
        tmp_opt_tbl.pcl_high_confidence_ppv_thres = nan;
        
        optimalPPV = nan;
        
        full_model_opt_score_thres = nan;

        tmp_opt_tbl.pcl_auc = nan;

        tmp_opt_tbl.pcl_opt_similarity_score_threshold = nan;
        tmp_opt_tbl.pcl_opt_ppv = nan;
        tmp_opt_tbl.pcl_opt_tpr = nan;
        tmp_opt_tbl.pcl_opt_fpr = nan;

    end
    
    all_trt_score_tbl = all_trt_score_tbl(all_trt_score_tbl.pcl_similarity_score >= highest_cmpd_min_pcl_similarity_score, :);

    full_model_next_score_moa = all_trt_score_tbl.target_description(full_model_next_score_idx);

    tmp_opt_tbl.next_score_moa = full_model_next_score_moa;

    %all_trt_score_tbl.full_model_opt_score_thres = repmat(full_model_opt_score_thres, [size(all_trt_score_tbl, 1) 1]);
    %all_trt_score_tbl.full_model_next_score_thres = repmat(full_model_next_score_thres, [size(all_trt_score_tbl, 1) 1]);
    %all_trt_score_tbl.full_model_next_score_ppv = repmat(full_model_next_score_ppv, [size(all_trt_score_tbl, 1) 1]);
    %all_trt_score_tbl.full_model_next_score_moa = repmat(full_model_next_score_moa, [size(all_trt_score_tbl, 1) 1]);
    
    all_trt_score_tbl.pcl_confidence_score = ppv_full; % PPV (Positive Predictive Value) taken as empirical PCL confidence scores at each observed PCL similarity score from KABX
    
    all_trt_score_tbl.rid = repmat(choose_pcl,[size(all_trt_score_tbl, 1) 1]);
    all_trt_score_tbl.pcl_desc = repmat(choose_pcl_desc,[size(all_trt_score_tbl, 1) 1]);


    out_tbl = [out_tbl; all_trt_score_tbl];

    tmp_opt_tbl = struct2table(tmp_opt_tbl);
    
    %tmp_opt_tbl.full_model_opt_score_thres = full_model_opt_score_thres;
    tmp_opt_tbl.full_model_next_score_thres = full_model_next_score_thres;
    tmp_opt_tbl.full_model_next_score_ppv = full_model_next_score_ppv;

    opt_tbl = [opt_tbl; tmp_opt_tbl];
    
    
    % test compounds section % 
    
    % Subset to test compound set rows for given PCL %
    test_idx = strcmp(ss_tbl_test_cmpd.rid, choose_pcl);
    sum(test_idx)

    ss_tbl_sele_test_cmpd = ss_tbl_test_cmpd(test_idx,:);
    size(ss_tbl_sele_test_cmpd)

    ss_tbl_sele_test_cmpd = sortrows(ss_tbl_sele_test_cmpd,{'pcl_similarity_score'},{'descend'});

    %%% Take similarity scores to this PCL for all test compound dsCGI profiles %%%
    
    all_trt_score_tbl_test_cmpd = ss_tbl_sele_test_cmpd(:, {'cid','pert_id','broad_id','proj_broad_id','target_description','pcl_similarity_score','rcorr','pcl_and_moa_agree'});
    
    %all_trt_score_tbl_test_cmpd.cmpd_index = repmat(nan,[size(all_trt_score_tbl_test_cmpd, 1) 1]);

    all_trt_scores_test_cmpd = all_trt_score_tbl_test_cmpd.pcl_similarity_score;
    all_trt_labels_test_cmpd = all_trt_score_tbl_test_cmpd.pcl_and_moa_agree;
    
    % all_trt_scores and ppv_full are from previous training step - similarity scores and confidence scores for all KABX dsCGI profiles to the PCL
    assert(length(all_trt_scores) == length(ppv_full), 'Length of KABX PCL similarity scores does not match length of PCL confidence scores');
    [score_thresholds, idx_unique] = unique(all_trt_scores, 'stable');
    
    % Estimate PCL confidence scores for each test compound dsCGI profile based on their PCL confidence score
    % by fitting to mapping learned from all KABX dsCGI profiles through linear interpolation which considers the distance (or difference)
    % an unknown dsCGI profile’s similarity score is to the two KABX dsCGI profile similarity scores it is between when estimating the confidence score
    test_ppv = interp1(score_thresholds, ppv_full(idx_unique), all_trt_scores_test_cmpd, 'linear');
    
    % Ensure no missing values
    test_ppv = fillmissing(test_ppv, 'nearest');

    assert(sum(isnan(test_ppv)) == 0)
    
    all_trt_score_tbl_test_cmpd.pcl_confidence_score = test_ppv; % Post-test probability / linear interpolation of fit PCL Similarity Score to PPV curve from previous steps
    
    all_trt_score_tbl_test_cmpd.rid = repmat(choose_pcl,[size(all_trt_score_tbl_test_cmpd, 1) 1]);
    all_trt_score_tbl_test_cmpd.pcl_desc = repmat(choose_pcl_desc,[size(all_trt_score_tbl_test_cmpd, 1) 1]);


    out_tbl_test_cmpd = [out_tbl_test_cmpd; all_trt_score_tbl_test_cmpd];

    
    % End test compounds section %

end

disp(i);

toc


num_pcls =

        1140

1/1140

choose_pcl =

  1x1 cell array

    {'30S ribosome:group1'}


jj =

     1


ans =

  4x1 cell array

    {'kabx2:BRD-K01825150-001-16-9:50.000000uM'}
    {'kabx2:BRD-A62336573-001-05-6:0.781250uM' }
    {'kabx2:BRD-K40939496-001-01-5:3.125000uM' }
    {'kabx2:BRD-K40939496-001-01-5:0.781250uM' }


this_pcl_size =

     4


ans =

        9427


ans =

        9427          36


ans =

   437


num_correct =

    25


auc =

    0.4376


optrocpt =

         0    0.0800


full_model_high_confidence_score_thres =

  single

    0.2180


full_model_next_score_thres =

  single

    0.2046


full_model_next_score_ppv =

  single

    0.6667


full_model_opt_score_thres =

  single

    0.2180


high_confidence_ppv_thres =

     1


optimalPPV =

     1


ans =

        1392


ans =

        1392          36


choose_pcl =

  1x1 cell array

    {'30S ribosome:group100'}


jj =

    21


ans =

  17x1 cell array

    {'screen6:BRD-M68646745-311-10-4:1.562

In [82]:
headt(out_tbl)


ans =

  11x3 table

    idx             field                                        value                           
    ___    ________________________    __________________________________________________________

     1     {'cid'                 }    {'kabx2:BRD-K40939496-001-01-5:0.781250uM'               }
     2     {'pert_id'             }    {'BRD-K40939496'                                         }
     3     {'broad_id'            }    {'BRD-K40939496-001-01-5'                                }
     4     {'proj_broad_id'       }    {'screen4wk_screen5wk_tbda1_kabx2:BRD-K40939496-001-01-5'}
     5     {'target_description'  }    {'30S ribosome'                                          }
     6     {'pcl_similarity_score'}    {[                                                0.2390]}
     7     {'rcorr'               }    {[                                                0.1136]}
     8     {'pcl_and_moa_agree'   }    {[                                                     1

In [83]:
headt(opt_tbl)


ans =

  16x3 table

    idx                          field                                    value         
    ___    __________________________________________________    _______________________

     1     {'pcl_rid'                                       }    {'30S ribosome:group1'}
     2     {'pcl_desc'                                      }    {'30S ribosome'       }
     3     {'pcl_n'                                         }    {[                437]}
     4     {'pcl_n_correct'                                 }    {[                 25]}
     5     {'pcl_n_trt'                                     }    {[               9427]}
     6     {'pcl_n_correct_trt'                             }    {[                477]}
     7     {'pcl_auc'                                       }    {[             0.4376]}
     8     {'pcl_high_confidence_similarity_score_threshold'}    {[             0.2180]}
     9     {'pcl_high_confidence_ppv_thres'                 }    {[                  1]

In [84]:
headt(out_tbl_test_cmpd)


ans =

  11x3 table

    idx             field                                   value                      
    ___    ________________________    ________________________________________________

     1     {'cid'                 }    {'tbda1:TCMDC-142724:0.097656uM'               }
     2     {'pert_id'             }    {'TCMDC-142724'                                }
     3     {'broad_id'            }    {'TCMDC-142724'                                }
     4     {'proj_broad_id'       }    {'screen4wk_screen5wk_tbda1_kabx2:TCMDC-142724'}
     5     {'target_description'  }    {'NA'                                          }
     6     {'pcl_similarity_score'}    {[                                      0.1790]}
     7     {'rcorr'               }    {[                                      0.1428]}
     8     {'pcl_and_moa_agree'   }    {[                                           0]}
     9     {'pcl_confidence_score'}    {[                                      0.2872]}
    10   

In [ ]:
% Save output tables as .txt files

wtable(out_tbl,fullfile(outdir,out_tbl_savename));

wtable(opt_tbl,fullfile(outdir,opt_tbl_savename));

wtable(out_tbl_test_cmpd,fullfile(outdir,out_tbl_test_cmpd_savename));

In [86]:
out_tbl_test_cmpd(1:30, :)


ans =

  30x11 table

                        cid                             pert_id             broad_id                         proj_broad_id                      target_description    pcl_similarity_score      rcorr      pcl_and_moa_agree    pcl_confidence_score              rid                  pcl_desc    
    ____________________________________________    ________________    ________________    ________________________________________________    __________________    ____________________    _________    _________________    ____________________    _______________________    ________________

    {'tbda1:TCMDC-142724:0.097656uM'           }    {'TCMDC-142724'}    {'TCMDC-142724'}    {'screen4wk_screen5wk_tbda1_kabx2:TCMDC-142724'}          {'NA'}                0.17895             0.14276            0                   0.28725          {'30S ribosome:group1'}    {'30S ribosome'}
    {'tbda1:TCMDC-142423:3.125000uM'           }    {'TCMDC-142423'}    {'TCMDC-142423'}    {'screen

In [87]:
full_model_next_score_moa


full_model_next_score_moa =

  1x1 cell array

    {'CysH'}



In [88]:
opt_tbl(1:30, :)


ans =

  3x16 table

             pcl_rid                 pcl_desc        pcl_n    pcl_n_correct    pcl_n_trt    pcl_n_correct_trt    pcl_auc    pcl_high_confidence_similarity_score_threshold    pcl_high_confidence_ppv_thres    pcl_opt_similarity_score_threshold    pcl_opt_ppv    pcl_opt_tpr    pcl_opt_fpr      next_score_moa      full_model_next_score_thres    full_model_next_score_ppv
    _________________________    ________________    _____    _____________    _________    _________________    _______    ______________________________________________    _____________________________    __________________________________    ___________    ___________    ___________    __________________    ___________________________    _________________________

    {'30S ribosome:group1'  }    {'30S ribosome'}     437          25            9427              477           0.43757                       0.21802                                      1                               0.21802            

In [89]:
opt_tbl(1, :)


ans =

  1x16 table

            pcl_rid                pcl_desc        pcl_n    pcl_n_correct    pcl_n_trt    pcl_n_correct_trt    pcl_auc    pcl_high_confidence_similarity_score_threshold    pcl_high_confidence_ppv_thres    pcl_opt_similarity_score_threshold    pcl_opt_ppv    pcl_opt_tpr    pcl_opt_fpr    next_score_moa    full_model_next_score_thres    full_model_next_score_ppv
    _______________________    ________________    _____    _____________    _________    _________________    _______    ______________________________________________    _____________________________    __________________________________    ___________    ___________    ___________    ______________    ___________________________    _________________________

    {'30S ribosome:group1'}    {'30S ribosome'}     437          25            9427              477           0.43757                       0.21802                                      1                               0.21802                       1  

In [91]:
out_tbl(1:30, 2:end)


ans =

  30x10 table

         pert_id                  broad_id                                   proj_broad_id                                                 target_description                           pcl_similarity_score      rcorr      pcl_and_moa_agree    pcl_confidence_score              rid                  pcl_desc    
    _________________    __________________________    __________________________________________________________    _______________________________________________________________    ____________________    _________    _________________    ____________________    _______________________    ________________

    {'BRD-K40939496'}    {'BRD-K40939496-001-01-5'}    {'screen4wk_screen5wk_tbda1_kabx2:BRD-K40939496-001-01-5'}    {'30S ribosome'                                               }          0.23901             0.11359            1                         1          {'30S ribosome:group1'}    {'30S ribosome'}
    {'BRD-K01825150'}    {'BRD-K01825150-001-1

In [92]:
out_tbl(height(out_tbl)-30:end, 4:end)


ans =

  31x8 table

                          proj_broad_id                                        target_description                 pcl_similarity_score      rcorr       pcl_and_moa_agree    pcl_confidence_score               rid                   pcl_desc    
    __________________________________________________________    ____________________________________________    ____________________    __________    _________________    ____________________    _________________________    ________________

    {'screen6:BRD-K57363465-001-01-5'                        }    {'PrrB'                                    }          -0.41756               0.276            0                  0.057208          {'30S ribosome:group102'}    {'30S ribosome'}
    {'screen6:BRD-K80404833-001-01-3'                        }    {'TrxC'                                    }          -0.42205           0.0026765            0                  0.057208          {'30S ribosome:group102'}    {'30S ribosome'}
    {

%ss_tbl_sele(1:5, {'broad_id', 'pcl_similarity_score', 'pcl_and_moa_agree'})

all_trt_score_tbl(1:30, 4:end)